<a href="https://colab.research.google.com/github/pamslover/pamellaPfe/blob/main/resumeparser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PACKAGE

In [ ]:
!pip install python-docx


In [ ]:
!pip install docx2txt


In [ ]:
!python -m spacy download fr_core_news_lg
!python -m spacy download en_core_web_lg

In [ ]:
!pip install pdfminer.six

In [ ]:
!pip install unidecode

In [ ]:

!pip install jsonlines

In [ ]:
!pip install yake

In [ ]:
!pip install textacy
!pip install langid

In [ ]:
# import spacy
# fr_nlp = spacy.load("fr_core_news_lg")
# en_nlp = spacy.load("en_core_web_lg")

In [ ]:
import pandas as pd
import csv
import numpy as np
import itertools
import os
import requests

# CLASS

## ExtractText and DataFrame

in this stage:
1. extract the text from the cvs words and PDF
2.normalize the text 
3.perform a first cleaning of our text
4.converting characters to lowercases.


In [ ]:
from io import StringIO
import glob
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from docx import Document
import docx2txt
import re


In [ ]:
import unicodedata


class extraction:


    def pdf_extract_text (filename, pages=None):
          if not pages:
            pagenums=set()
          else:
            pagenums=set(pages)

          contenue=[]

          with open (filename, 'rb') as in_file:
            if filename.endswith ('.docx') or filename.endswith ('.doc'):
              text=docx2txt.process(in_file)
              text=unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8') 
              contenue= ' '.join([str(elem) for elem in (text.split('\n'))])
            elif filename.endswith ('.pdf'):
              parser= PDFParser(in_file)
              doc=PDFDocument(parser)
              output= StringIO()
              manager= PDFResourceManager()
              converter= TextConverter(manager, output, laparams=LAParams())
              interpreter= PDFPageInterpreter (manager, converter)
              for page in PDFPage.get_pages(in_file, pagenums):
                interpreter.process_page(page)
              text=output.getvalue()
              text= unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('UTF-8') 
              contenue= ' '.join([str(elem) for elem in (text.splitlines())])
              in_file.close()
              converter.close()
              output.close()
            else:
              pass
          return  contenue

    def extract_number(text):
          numbers=[]
          phone=(re.findall((r'[0-9]?[0-9 (\)]{8,15}[0-9]') ,str(text)))
          if phone:
                    number= ''.join(phone[0])
                    if text.find(number)>=0 and len(number)<20:
                      numbers.append(''.join(str(elem) for elem in number))
          return numbers

    def extract_date (text):
          dates=[]
          dates.append(re.findall((r"\d{1,2}[-/]\d{1,2}[-/]\d{2,4}") ,text))
          dates.append(re.findall((r"\w{4,9} ?\d{2,4}") ,text))
          dates.append(re.findall((r"\d{2,4}[ ][-/][ ]\d{2,4}") ,text))
          
          return dates


    # def extract_name(text):
    #   nlp_text = nlp(text)
    #   # First name and Last name are always Proper Nouns
    #   pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    #   matcher.add('NAME', [pattern], on_match = None)
    #   matches = matcher(nlp_text)
      
    #   for match_id, start, end in matches:
    #       span = nlp_text[start:end]
    #       return span.text


  


In [ ]:
import unidecode

class cleaner:

    def __init__(self) -> None:
        pass
    
   
    def clean_text(sentence:str):
        
        text=unicodedata.normalize('NFKD', sentence).encode('ascii', 'ignore').decode('utf8') 
        text =cleaner.remove_special_char(cleaner.extract_only_text(text))
        return cleaner.remove_double_space(text)
    
    def remove_double_space(text):
         return re.sub(r'([\s])\1{1,}|([\.])\1{1,}', r'\1', str(text))
        #  return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8')

    def extract_only_text(text):
        text= text.lower()
        text= re.sub(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+', ' ', str(text))
        text= re.sub(r'[^ ]+\.[^ ]+', ' ', str(text))
        # text = re.sub('\S*www\S*\s?', ' ', str(text))
        text = re.sub('http\S+\s*', ' ', str(text))
        # text = re.sub('(\d+\.)+\d*.*','',str(text))
        text = re.sub('#S+', ' ', str(text))  # remove hashtags
        text = re.sub('@S+', '  ', str(text))  # remove mentions
        text = re.sub('\s+', ' ', str(text))  # remove extra whitespace
        text = re.sub("\t",' ',str(text))
        text = re.sub('\n|\r',' ',str(text))
        text = re.sub('[^\x00-\x7f]',' ',str(text))
        text= re.sub(r'\x00|\uf001','fi',str(text))
        text= re.sub(r"cv|'",' ',str(text))
        text = re.sub(r'@+[A-Za-z0-9]+', ' ', str(text)) 
        text= re.sub(r'\x0c',' ',str(text))
        text= re.sub(r'www[a-z0-9\.\-+_]+\.[a-z]+[/-][a-zA-Z]?[a-z\-]+', r' ', str(text))
        text = text.replace('\\xa0', ' ')  
        # text= re.sub(r'\uf005|\uf006|\uf10b|\ue81e|\ue811|\ue80d|\ue81a|\ue803', r' ', str(text))
      
        return text
    
    def remove_special_char(text:str):
        return re.sub(r"[^a-zA-Z0-9\-“”\s\'\.]"," ", str(text))



In [ ]:
from sys import path
from pathlib import Path


class Data:


    def dataframe():

        liste_test = glob.glob ('/content/drive/MyDrive/projet pfe/data/*')
        liste=[]
        title=[]
        Email=[]
        number=[]
        for filename in liste_test:
                # text = pdf_extract_text (filename)
                text = extraction.pdf_extract_text (filename).lower()
                texte=cleaner.clean_text(text)
                liste.append(texte)
                email= (re.findall((r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+'), str(text)))
                Email.append( ' '.join([str(elem) for elem in email]))
                number.append( extraction.extract_number(text))
                # liste.append(text)
                title.append(Path(filename).stem)


        data=pd.DataFrame(liste, index=None)
        data['Candidates']= pd.DataFrame(title)
        data['Emails']= pd.DataFrame(Email)
        data['Phone_number']= pd.DataFrame(number)
        data['Resumes']= pd.DataFrame(liste)

        data= data.drop(columns=0)

        data.to_csv('data_cv.csv') 
          
        df = pd.read_csv('data_cv.csv', engine='c' ) 
        df= df.drop(columns='Unnamed: 0')
        return df
        

In [ ]:
df= Data.dataframe()


In [ ]:
jd = pd.read_csv('/content/drive/MyDrive/projet pfe/JobDescription.csv')
# 

In [ ]:
jd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/projet pfe/data_cv.csv')
df = df.drop(columns='Unnamed: 0')

In [ ]:
jd['Bullets'] = jd.Bullets.apply(lambda x: cleaner.clean_text(sentence = x))
jd['Title'] = jd.Title.apply(lambda x: cleaner.clean_text(sentence = x))

In [ ]:

# jd = jd.drop(columns='Unnamed: 0')
jd['Bullets'][1]

In [ ]:
df

In [ ]:
resumes = df['Resumes'].astype('str').tolist()
jobs = jd['Bullets'].astype('str').tolist

## Preprocessing

In this stage, I followed basic cleaning processes for text analysis which includes:

1.remove punctuations.

2.tokenize word and sentence.

3.remove stopwords

4.part of speech and lemmatization

5.lang detection

In [ ]:
from langid.langid import LanguageIdentifier, model
import langid


class LanguageCheck:

    # def __init__(self):
    #     self.text = text
    #     self.language = None

    def override(self, text):
        self.text = text
        identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
        self.language, _ = identifier.classify(self.text)

        if self.language == "fr":
          self.language = "french"
        elif self.language == "en":
          self.language = "english"


        return self.language


In [ ]:

from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

class process:


    def __init__(self):
      self.lang,_= LanguageCheck().override(text)
      _,self.doc = LanguageCheck().override(text)


    def _segment_sentences(self,text):

        text = self._clean_sentences(text)
        #text = ProcessText.remove_double_space(text.replace("\t"," ")) #replace("\n"," ").
        return [ sentence.text for sentence in self.doc.sents if len(sentence)> 1]


    def _clean_sentences(self,sentences):

        for i,sent in enumerate(sentences):
            sentence = cleaner.clean_text(sent)
            sentence = cleaner.remove_double_space(sentence.replace("\n"," "))
            if len(sentence) > 1:
                sentences[i] = sentence

        return sentences

    def _tokenization(self, text, all_pos= False):

        sentences = self._segment_sentences(text)
        if sentences!="":
            text = " ".join(sentences)
        if not all_pos:
            pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB', "ADV", "X"]  
            return [token for token in self.doc if not token.is_stop 
                    and len(token)>1 
                    and token.pos_ in pos_tag]

        return[token for token in self.doc if not token.is_stop and len(token)>1 ]


    def _token(self,text):

        tok=[token.text.lower() for token in self.doc 
                                      if len(token)>1 and token.isalpha() ]
        return tok

    
    def tokens(self,text):

        tokens = self._tokenization(text)
        return self._token(tokens)


    def lemmatization (self,text): 
        return [tok.lemma_ for tok in self.tokens(text)]

""" nous avons ajouté les stopwords pour la verfication de similitude entre les cv et offres et tester si l'ajout des stopwords à une importance ou non"""

    def stopword(text):
      identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
      lang, _ = identifier.classify(text)
       if lang == "fr":
          stopWords = set(stopwords.words('french'))
          words = word_tokenize(text)
          wordsFiltered = []

          for w in words:
              if w not in stopWords:
                  wordsFiltered.append(w)
       
       if lan == "en":
          stopWords = set(stopwords.words('english'))
          words = word_tokenize(text)
          wordsFiltered = []

          for w in words:
              if w not in stopWords:
                  wordsFiltered.append(w)
              
      return wordsFiltered







In [ ]:
process().tokens(text)

In [ ]:
output_string = StringIO()
with open('/content/drive/MyDrive/projet pfe/data/C.V.pdf', 'rb') as in_file:
    parser = PDFParser(in_file)
    docu = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(docu):
        interpreter.process_page(page)
    text= output_string.getvalue()
    text=unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8') 
    text= ' '.join([elem for elem in (text.split('\n'))])
    in_file.close()
    device.close()
    output_string.close()


## evaluator



In [ ]:
class Keywords:

    def __init__(self):
        '''
        this is the test of my bla bla car
        '''
        self.text = None
        self.lang = "french"
        self.windowSize = 1
        self.array_of_keywords = []
        self.numOfKeywords = 50
        self.max_ngram_size = 3
        self.deduplication_threshold = 0.9
        self.deduplication_algo = 'seqm'

    def override(self, text=None, language="french", num_of_keywords=50, max_ngram_size=3, deduplication_thresold=0.9):
        self.text = text
        self.lang = language
        self.numOfKeywords = num_of_keywords
        self.max_ngram_size = max_ngram_size
        self.deduplication_threshold = deduplication_thresold
        custom_kw_extractor = yake.KeywordExtractor(lan=self.lang, n=self.max_ngram_size,
                                                    dedupLim=self.deduplication_threshold,
                                                    dedupFunc=self.deduplication_algo, windowsSize=self.windowSize,
                                                    top=self.numOfKeywords, features=None)
        self.array_of_keywords = custom_kw_extractor.extract_keywords(text)
        return self.array_of_keywords

In [ ]:
import yake
from yake.highlight import TextHighlighter


class Keywords:

    def __init__(self):
        '''
        this is the test of my bla bla car
        '''
        self.text = None
        self.lang = "fr"
        self.windowSize = 1
        self.array_of_keywords = []
        self.numOfKeywords = 30
        self.max_ngram_size = 3
        self.deduplication_threshold = 0.9
        self.deduplication_algo = 'seqm'

    def override(self, text=None, language="en", num_of_keywords=30, max_ngram_size=3, deduplication_thresold=0.9):
        self.text = text
        self.lang = language
        self.numOfKeywords = num_of_keywords
        self.max_ngram_size = max_ngram_size
        self.deduplication_threshold = deduplication_thresold
        custom_kw_extractor = yake.KeywordExtractor(lan=self.lang, n=self.max_ngram_size,
                                                    dedupLim=self.deduplication_threshold,
                                                    dedupFunc=self.deduplication_algo, windowsSize=self.windowSize,
                                                    top=self.numOfKeywords, features=None)
        self.array_of_keywords = custom_kw_extractor.extract_keywords(text)
        return self.array_of_keywords




## Vectorisation

ici, nous allons tester une ensemble d'algo pour la vectorisation de nos cv et offre d'emplois tel que:

*word2vec

*tf-idf

*bac of words

et bien d'autres ainsi nous choisirons celui qui s'adapte le mieux à notre probléme.


In [ ]:

!pip install --upgrade gensim

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

corpus= df["Resumes"]
vectorizer= TfidfVectorizer()
vector=vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
dense = vector.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df.index = ([f"TFIDF_{i}" for i in range(1,len(corpus)+1)])
print(df.head(10))



In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


#on peut utiliser la similarité entre un cv et une offre d'emplois pour ceterminer à quel pourcentage ils sont proche ensuite recuperer 
# les cv avec un grand pourcentage et les passer ensuite dans reseau de neurone


class Evaluator:

    def __init__(self):
      self.doc= process.Doc()

    def computeTFIDF(self, corpus):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform(corpus)
        feature_names = vectorizer.get_feature_names()
        dense = vectors.todense()
        denselist = dense.tolist()
        df = pd.DataFrame(denselist, columns=feature_names)
        df.index = ([f"TFIDF_{i}" for i in range(1,len(corpus)+1)])

        return df.T



    def countvectorizer(self,corpus):
      countvectorizer= CountVectorizer()
      X=countvectorizer.fit_transform(corpus)
      result= X.toarray()
      return result


    def similarity(self,corpus):
        similarities = {}
        for i,document in enumerate(corpus,1):
            doc=self.doc(document)
            sim = []
            for document_ in corpus:
                sim.append(doc.similarity(self.doc(document_))) 
            similarities[f"Doc_{i}"] = sim
        df = pd.DataFrame(similarities, index=[f"Doc_{i}" for i in range(1,len(corpus)+1)])
        return df


    def display_lda(self,lda_model, corpus, id2word):
        return
        pyLDAvis.enable_notebook()
        vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
        return vis


    def LDA(self,data_words):
        id2word = corpora.Dictionary(data_words)
        corpus = []
        
        for text in data_words:
            new = id2word.doc2bow(text)
            corpus.append(new)
        num_topic = len(corpus)
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topic,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           eta = [.01]*len(id2word.keys()),
                                           alpha=[.01]*num_topic)

        return lda_model ,self.display_lda(lda_model,corpus,id2word)

In [ ]:
import gensim.downloader as api
from gensim.test.utils import datapath
from gensim import utils

wv = api.load('word2vec-google-news-300')


In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(min_count=1)
model.build_vocab(sentences= sentences)  
model.train(sentences= sentences, total_examples=model.corpus_count, epochs=model.epochs)  


# model  word2vec avec Gensing

In [ ]:
import pandas as pd
from gensim.summarization import keywords
import matplotlib.pyplot as plt
# df =pd.read_csv('data.csv')
job = jd['Bullets'].tolist()
resume = df['Resumes'].tolist()
# positions = df['position'].tolist()

from gensim import models
docs = []
for i in range(len(jd)):
    sent = models.doc2vec.LabeledSentence(words = job[i].split(),tags = ['{}_{}'.format(job[i], i)])
    #sent = models.doc2vec.LabeledSentence(words = jd[i].split(),tags = ['jd'])
    docs.append(sent)
    
model = models.Doc2Vec(alpha=.025, min_alpha=.025, min_count=1)
model.build_vocab(docs)

for epoch in range(10):
    model.train(docs)
    model.alpha -= 0.002  # decrease the learning rate`
    model.min_alpha = model.alpha  # fix the learning rate, no decay
    
with open('resumeconverted.txt','r') as f:
    resume = f.read()

    
from sklearn.manifold import MDS
data = []
for i in range(len(jd)):
    data.append(model.docvecs[i])

data.append(model.infer_vector(resume))

mds = MDS(n_components=2, random_state=1)
pos = mds.fit_transform(data)
#print pos
xs,ys = pos[:,0], pos[:,1]
for x, y in zip(xs, ys):
    plt.scatter(x, y)
#    plt.text(x, y, name)
xs2,ys2 = xs[-1], ys[-1]
plt.scatter(xs2, ys2, c='Red', marker='+')
plt.text(xs2,ys2,'resume')
plt.savefig('distance.png')
plt.show()


from sklearn.metrics.pairwise import cosine_distances
cos_dist =[]
for i in range(len(data)-1):
    print(i)
    cos_dist.append(float(cosine_distances(model.infer_vector(resume),data[i])))
    

key_list =[]

for j in resume:
    key =''
    for word in keywords(j).split('\n'):
        key += '{} '.format(word)
    key_list.append(key)

summary = pd.DataFrame({
        'Resumes': resume,
        # 'Postition': positions,
        'Cosine Distances': cos_dist,
        'Keywords': key_list,
        'Job Description': job
    })
z =summary.sort('Cosine Distances', ascending=False)
z.to_csv('Summary.csv',encoding="utf-8")

In [ ]:
print(docs)